In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip uninstall -y kaggle
!pip install kaggle
import os
import zipfile

os.environ["KAGGLE_USERNAME"] = "mizanurr"
os.environ["KAGGLE_KEY"] = "654c2a7dc145088aa93c65566a027cf6"
!kaggle competitions download -c stanford-covid-vaccine
!kaggle datasets download -d vaghefi/aug-vaccine


Uninstalling kaggle-1.5.8:
  Successfully uninstalled kaggle-1.5.8
Processing /root/.cache/pip/wheels/94/a7/09/68dc83c7c14fdbdf5d3f2b2da5b87e587bfc1e85df69b1130c/kaggle-1.5.8-cp36-none-any.whl
stanford-covid-vaccine.zip: Skipping, found more recently modified local copy (use --force to force download)
 80% 73.0M/91.1M [00:04<00:01, 10.1MB/s]
100% 91.1M/91.1M [00:04<00:00, 22.3MB/s]


In [3]:
!mkdir data

In [4]:
with zipfile.ZipFile('/content/stanford-covid-vaccine.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data/')

In [5]:
with zipfile.ZipFile('/content/aug-vaccine.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/data/')

In [6]:
!pip install spektral

     |████████████████████████████████| 102kB 2.6MB/s 


In [7]:
#the basics
import pandas as pd 
import numpy as np
import seaborn as sns
import math, json
from matplotlib import pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split, KFold
import time
from spektral.layers import GraphAttention, GatedGraphConv

#tensorflow deep learning basics
import tensorflow as tf

print('tf version:',tf.__version__)

print('\nList of GPUs:')
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

tf version: 2.3.0

List of GPUs:
Name: /physical_device:GPU:0   Type: GPU


In [8]:
start = time.time()

import os
folder = '/content/data/bpps'
files = os.listdir(folder)

lst = []
for file in tqdm(files):
    data = np.load(os.path.join(folder, file))
    lst.append([file.replace('.npy',''), data])
    
bpps = pd.DataFrame(lst, columns=['id', 'bpps'])

100%|██████████| 6034/6034 [00:02<00:00, 2959.69it/s]


In [9]:
def get_structure_feature(structure):
    pm = np.zeros((len(structure), 3))
    start_token_indices = []
    for i, token in enumerate(structure):
        if token == "(":
            start_token_indices.append(i)
        elif token == ")":
            j = start_token_indices.pop()
            pm[i, 0] = pm[i, 0] = i - j
            pm[i, 1] = pm[j, 1] = structure[j:i+1].count('.')
            pm[i, 2] = pm[j, 2] = structure[j:i+1].count(')') - 1
    return pm

In [10]:
train = pd.read_json('/content/data/train.json', lines=True).merge(bpps)
test = pd.read_json('/content/data/test.json', lines=True).merge(bpps)
sample_sub = pd.read_csv('/content/data/sample_submission.csv')
augment = pd.read_csv('/content/drive/My Drive/Vaccine/aug_data1.csv')

In [11]:
# train = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True).merge(bpps)
# test = pd.read_json('../input/stanford-covid-vaccine/test.json', lines=True).merge(bpps)
# sample_sub = pd.read_csv('../input/stanford-covid-vaccine/sample_submission.csv')
# augment = pd.read_csv('../input/aug-vaccine/aug_data1.csv')


train_aug = train.drop(columns=['structure', 'predicted_loop_type']).merge(augment, how='left', on=['id','sequence'])
test_aug = test.drop(columns=['structure', 'predicted_loop_type']).merge(augment, how='left', on=['id','sequence'])

test = test.append(test_aug[test.columns])

#target columns
target_cols = ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C', 'deg_50C']
error_cols = ['reactivity_error', 'deg_error_Mg_pH10', 'deg_error_pH10', 'deg_error_Mg_50C', 'deg_error_50C']
metric_cols = [0, 1, 3]

token = {'sequence': 'ACGU', 'structure': '().', 'predicted_loop_type': 'BEHIMSX'}
token_map = {key: {x:i for i, x in enumerate(val)} for key, val in token.items()}

def get_adj(structure):
    pm = np.zeros((len(structure), len(structure)))
    start_token_indices = []
    for i, token in enumerate(structure):
        if token == "(":
            start_token_indices.append(i)
        elif token == ")":
            j = start_token_indices.pop()
            pm[i, j] = 1.0
            pm[j, i] = 1.0
        pm[i, i] = 1
    return pm


mu_bpps = 0.368
sd_bpps = 0.415

def preprocess_inputs(df):
    features = []
    for col in ['sequence', 'structure', 'predicted_loop_type']:
        x = np.array(df[col].apply(lambda seq: [token_map[col][x] for x in seq]).values.tolist())
        features.append(tf.keras.backend.one_hot(x,len(token_map[col])))
    bpps = np.array(df['bpps'].tolist())
    features.append((1-bpps.sum(-1))[:,:,None])
    features.append(bpps.max(-1)[:,:,None])
    features.append((bpps > 0).mean(-1)[:,:,None])
    features.append(np.array(df['structure'].apply(get_structure_feature).values.tolist())/130)
    return np.concatenate(features, axis=-1)

print('processing train')
train_snfilter = train.SN_filter.values
train_inputs = preprocess_inputs(train)
train_inputs = np.pad(train_inputs,((0, 0), (0, 130-107), (0, 0)))

train_inputs_aug = preprocess_inputs(train_aug)
train_inputs_aug = np.pad(train_inputs_aug,((0, 0), (0, 130-107), (0, 0)))
train_ids_aug = train_aug['id'].values

train_weights = np.array(train.signal_to_noise.values.tolist())
train_weights_aug = np.array(train_aug.signal_to_noise.values.tolist())

train_labels = np.array(train[target_cols].values.tolist()).transpose((0, 2, 1))
train_labels_aug = np.array(train_aug[target_cols].values.tolist()).transpose((0, 2, 1))

print('Original Train:', train_inputs.shape, train_labels.shape, train_weights.shape)
print('Augmented Train:', train_inputs_aug.shape, train_labels_aug.shape, train_weights_aug.shape)

train_adj = np.array(train['structure'].apply(get_adj).values.tolist())
train_adj = np.pad(train_adj, ((0,0),(0,130-107),(0,130-107)), 'constant')
train_adj_aug = np.array(train_aug['structure'].apply(get_adj).values.tolist())
train_adj_aug = np.pad(train_adj_aug, ((0,0),(0,130-107),(0,130-107)), 'constant')

processing train
Original Train: (2400, 130, 20) (2400, 68, 5) (2400,)
Augmented Train: (2400, 130, 20) (2400, 68, 5) (2400,)


In [12]:
#get different test sets and process each
public_df = test.query("seq_length == 107").copy()
private_df = test.query("seq_length == 130").copy()

print('processing test')
public_inputs = preprocess_inputs(public_df)
public_inputs = np.pad(public_inputs,((0, 0), (0, 130-public_inputs.shape[1]), (0, 0)))

private_inputs = preprocess_inputs(private_df)
private_inputs = np.pad(private_inputs,((0, 0), (0, 130-private_inputs.shape[1]), (0, 0)))

public_adj = np.array(public_df.loc[:, 'structure'].apply(get_adj).values.tolist())
public_adj = np.pad(public_adj, ((0,0),(0,130-public_adj.shape[1]),(0,130-public_adj.shape[1])), 'constant')

private_adj = np.array(private_df.loc[:, 'structure'].apply(get_adj).values.tolist())
private_adj = np.pad(private_adj, ((0,0),(0,130-private_adj.shape[1]),(0,130-private_adj.shape[1])), 'constant')

print('Test Sizes:', public_inputs.shape, public_adj.shape, private_inputs.shape, private_adj.shape)


processing test
Test Sizes: (1258, 130, 20) (1258, 130, 130) (6010, 130, 20) (6010, 130, 130)


In [13]:
public_preds = np.load('/content/data/gtn_aug_snr_gru_public_preds.npy')
public_std = public_preds.mean(3).mean(2).std(0)
good_public = np.where(public_std < np.quantile(public_std, 0.13))[0]
public_labels = public_preds.mean(0)[good_public][:,:68,:]
print('Public Pseudo-labeling Sizes:', public_labels.shape)

Public Pseudo-labeling Sizes: (164, 68, 5)


In [14]:
private_preds = np.load('/content/data/gtn_aug_snr_gru_private_preds.npy')
private_std = private_preds.mean(3).mean(2).std(0)
good_private = np.where(private_std < np.quantile(private_std, 0.13))[0]
private_labels = private_preds.mean(0)[good_private][:,:68,:]
print('Private Pseudo-labeling Sizes:', private_labels.shape)

Private Pseudo-labeling Sizes: (782, 68, 5)


In [15]:
""" TF-Keras SWA: callback utility for performing stochastic weight averaging (SWA).
"""

import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import BatchNormalization

class SWA(Callback):
    """ Stochastic Weight Averging.
    # Paper
        title: Averaging Weights Leads to Wider Optima and Better Generalization
        link: https://arxiv.org/abs/1803.05407
    # Arguments
        start_epoch:   integer, epoch when swa should start.
        lr_schedule:   string, type of learning rate schedule.
        swa_lr:        float, learning rate for swa sampling.
        swa_lr2:       float, upper bound of cyclic learning rate.
        swa_freq:      integer, length of learning rate cycle.
        verbose:       integer, verbosity mode, 0 or 1.
    """
    def __init__(self,
                 start_epoch,
                 no_samples,
                 lr_schedule='manual',
                 swa_lr='auto',
                 swa_lr2='auto',
                 swa_freq=1,
                 verbose=0):
                 
        super(SWA, self).__init__()
        self.start_epoch = start_epoch - 1
        self.lr_schedule = lr_schedule
        self.swa_lr = swa_lr
        self.swa_lr2 = swa_lr2
        self.swa_freq = swa_freq
        self.verbose = verbose
        self.no_samples = no_samples
        # self.params['samples'] = no_samples 

        if start_epoch < 2:
            raise ValueError('"swa_start" attribute cannot be lower than 2.')

        schedules = ['manual', 'constant', 'cyclic']

        if self.lr_schedule not in schedules:
            raise ValueError('"{}" is not a valid learning rate schedule' \
                             .format(self.lr_schedule))

        if self.lr_schedule == 'cyclic' and self.swa_freq < 2:
            raise ValueError('"swa_freq" must be higher than 1 for cyclic schedule.')

        if self.swa_lr == 'auto' and self.swa_lr2 != 'auto':
            raise ValueError('"swa_lr2" cannot be manually set if "swa_lr" is automatic.') 
            
        if self.lr_schedule == 'cyclic' and self.swa_lr != 'auto' \
           and self.swa_lr2 != 'auto' and self.swa_lr > self.swa_lr2:
            raise ValueError('"swa_lr" must be lower than "swa_lr2".')

    def on_train_begin(self, logs=None):

        self.epochs = self.params.get('epochs')

        if self.start_epoch >= self.epochs - 1:
            raise ValueError('"swa_start" attribute must be lower than "epochs".')

        self.init_lr = K.eval(self.model.optimizer.lr)

        # automatic swa_lr
        if self.swa_lr == 'auto':
            self.swa_lr = 0.1*self.init_lr
        
        if self.init_lr < self.swa_lr:
            raise ValueError('"swa_lr" must be lower than rate set in optimizer.')

        # automatic swa_lr2 between initial lr and swa_lr   
        if self.lr_schedule == 'cyclic' and self.swa_lr2 == 'auto':
            self.swa_lr2 = self.swa_lr + (self.init_lr - self.swa_lr)*0.25

        self._check_batch_norm()

    def on_epoch_begin(self, epoch, logs=None):

        self.current_epoch = epoch
        self._scheduler(epoch)

        # constant schedule is updated epoch-wise
        if self.lr_schedule == 'constant' or self.is_batch_norm_epoch:
            self._update_lr(epoch)

        if self.is_swa_start_epoch:
            self.swa_weights = self.model.get_weights()

            if self.verbose > 0:
                print('\nEpoch %05d: starting stochastic weight averaging'
                      % (epoch + 1))

        if self.is_batch_norm_epoch:
            self._set_swa_weights(epoch)

            if self.verbose > 0:
                print('\nEpoch %05d: reinitializing batch normalization layers'
                      % (epoch + 1))

            self._reset_batch_norm()

            if self.verbose > 0:
                print('\nEpoch %05d: running forward pass to adjust batch normalization'
                      % (epoch + 1))

    def on_batch_begin(self, batch, logs=None):

        # update lr each batch for cyclic lr schedule
        if self.lr_schedule == 'cyclic':
            self._update_lr(self.current_epoch, batch)

        if self.is_batch_norm_epoch:
            batch_size = 64
            momentum = batch_size / (batch*batch_size + batch_size)

            for layer in self.batch_norm_layers:
                layer.momentum = momentum

    def on_batch_end(self, batch, logs=None):
        logs = logs or {}
        logs['lr'] = K.eval(self.model.optimizer.lr)
        for k, v in logs.items():
            if k == 'lr':
                self.model.history.history.setdefault(k, []).append(v)

    def on_epoch_end(self, epoch, logs=None):

        if self.is_swa_start_epoch:
            self.swa_start_epoch = epoch

        if self.is_swa_epoch and not self.is_batch_norm_epoch:
            self.swa_weights = self._average_weights(epoch)

    def on_train_end(self, logs=None):

        if not self.has_batch_norm:
            self._set_swa_weights(self.epochs)
        else:
            self._restore_batch_norm()

    def _scheduler(self, epoch):

        swa_epoch = (epoch - self.start_epoch)

        self.is_swa_epoch = epoch >= self.start_epoch and swa_epoch % self.swa_freq == 0
        self.is_swa_start_epoch = epoch == self.start_epoch
        self.is_batch_norm_epoch = epoch == self.epochs - 1 and self.has_batch_norm

    def _average_weights(self, epoch):

        return [(swa_w * (epoch - self.start_epoch) + w)
                / ((epoch - self.start_epoch) + 1)
                for swa_w, w in zip(self.swa_weights, self.model.get_weights())]

    def _update_lr(self, epoch, batch=None):

        if self.is_batch_norm_epoch:
            lr = 0
            K.set_value(self.model.optimizer.lr, lr)
        elif self.lr_schedule == 'constant':
            lr = self._constant_schedule(epoch)
            K.set_value(self.model.optimizer.lr, lr)
        elif self.lr_schedule == 'cyclic':
            lr = self._cyclic_schedule(epoch, batch)
            K.set_value(self.model.optimizer.lr, lr)

    def _constant_schedule(self, epoch):

        t = epoch / self.start_epoch
        lr_ratio = self.swa_lr / self.init_lr
        if t <= 0.5:
            factor = 1.0
        elif t <= 0.9:
            factor = 1.0 - (1.0 - lr_ratio) * (t - 0.5) / 0.4
        else:
            factor = lr_ratio
        return self.init_lr * factor

    def _cyclic_schedule(self, epoch, batch):
        """ Designed after Section 3.1 of Averaging Weights Leads to
        Wider Optima and Better Generalization(https://arxiv.org/abs/1803.05407)
        """
        # steps are mini-batches per epoch, equal to training_samples / batch_size
        steps = self.params.get('steps')
        
        #occasionally steps parameter will not be set. We then calculate it ourselves
        if steps == None:
            self.params['samples'] = self.no_samples
            steps = self.params['samples'] // self.params['batch_size']
        
        swa_epoch = (epoch - self.start_epoch) % self.swa_freq
        cycle_length = self.swa_freq * steps

        # batch 0 indexed, so need to add 1
        i = (swa_epoch * steps) + (batch + 1)
        if epoch >= self.start_epoch:
            t = (((i-1) % cycle_length) + 1)/cycle_length
            return (1-t)*self.swa_lr2 + t*self.swa_lr
        else:
            return self._constant_schedule(epoch)

    def _set_swa_weights(self, epoch):

        self.model.set_weights(self.swa_weights)

        if self.verbose > 0:
            print('\nEpoch %05d: final model weights set to stochastic weight average'
                  % (epoch + 1))

    def _check_batch_norm(self):

        self.batch_norm_momentums = []
        self.batch_norm_layers = []
        self.has_batch_norm = False
        self.running_bn_epoch = False

        for layer in self.model.layers:
            if issubclass(layer.__class__, BatchNormalization):
                self.has_batch_norm = True
                self.batch_norm_momentums.append(layer.momentum)
                self.batch_norm_layers.append(layer)

        if self.verbose > 0 and self.has_batch_norm:
            print('Model uses batch normalization. SWA will require last epoch '
                  'to be a forward pass and will run with no learning rate')

    def _reset_batch_norm(self):

        for layer in self.batch_norm_layers:

            # to get properly initialized moving mean and moving variance weights
            # we initialize a new batch norm layer from the config of the existing
            # layer, build that layer, retrieve its reinitialized moving mean and
            # moving var weights and then delete the layer
            bn_config = layer.get_config()
            new_batch_norm = BatchNormalization(**bn_config)
            new_batch_norm.build(layer.input_shape)
            new_moving_mean, new_moving_var = new_batch_norm.get_weights()[-2:]
            # get rid of the new_batch_norm layer
            del new_batch_norm
            # get the trained gamma and beta from the current batch norm layer
            trained_weights = layer.get_weights()
            new_weights = []
            # get gamma if exists
            if bn_config['scale']:
                new_weights.append(trained_weights.pop(0))
            # get beta if exists
            if bn_config['center']:
                new_weights.append(trained_weights.pop(0))
            new_weights += [new_moving_mean, new_moving_var]
            # set weights to trained gamma and beta, reinitialized mean and variance
            layer.set_weights(new_weights)

    def _restore_batch_norm(self):

        for layer, momentum in zip(self.batch_norm_layers, self.batch_norm_momentums):
            layer.momentum = momentum

In [16]:
import tensorflow.keras.layers as L
import keras.backend as K
import tensorflow as tf
import plotly.express as px
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.cluster import KMeans

In [17]:
class MultiHeadSelfAttention(L.Layer):
    def __init__(self, embed_dim, num_heads=8):
        super(MultiHeadSelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.num_heads = num_heads
        if embed_dim % num_heads != 0:
            raise ValueError(
                f"embedding dimension = {embed_dim} should be divisible by number of heads = {num_heads}"
            )
        self.projection_dim = embed_dim // num_heads
        self.query_dense = L.Dense(embed_dim)
        self.key_dense = L.Dense(embed_dim)
        self.value_dense = L.Dense(embed_dim)
        self.combine_heads = L.Dense(embed_dim)

    def attention(self, query, key, value):
        score = tf.matmul(query, key, transpose_b=True)
        dim_key = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_score = score / tf.math.sqrt(dim_key)
        weights = tf.nn.softmax(scaled_score, axis=-1)
        output = tf.matmul(weights, value)
        return output, weights

    def separate_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.projection_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        # x.shape = [batch_size, seq_len, embedding_dim]
        batch_size = tf.shape(inputs)[0]
        query = self.query_dense(inputs)  # (batch_size, seq_len, embed_dim)
        key = self.key_dense(inputs)  # (batch_size, seq_len, embed_dim)
        value = self.value_dense(inputs)  # (batch_size, seq_len, embed_dim)
        query = self.separate_heads(
            query, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        key = self.separate_heads(
            key, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        value = self.separate_heads(
            value, batch_size
        )  # (batch_size, num_heads, seq_len, projection_dim)
        attention, weights = self.attention(query, key, value)
        attention = tf.transpose(
            attention, perm=[0, 2, 1, 3]
        )  # (batch_size, seq_len, num_heads, projection_dim)
        concat_attention = tf.reshape(
            attention, (batch_size, -1, self.embed_dim)
        )  # (batch_size, seq_len, embed_dim)
        output = self.combine_heads(
            concat_attention
        )  # (batch_size, seq_len, embed_dim)
        return output

In [18]:
class EncoderBlock(L.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.3):
        super(EncoderBlock, self).__init__()
        self.att = MultiHeadSelfAttention(embed_dim, num_heads)
        self.ffn = tf.keras.Sequential(
            [L.Dense(ff_dim, activation="relu"), L.Dense(embed_dim),]
        )
        self.layernorm1 = L.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = L.LayerNormalization(epsilon=1e-6)
        self.dropout1 = L.Dropout(rate)
        self.dropout2 = L.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [19]:
class DecoderBlock(L.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.3):
        super(DecoderBlock, self).__init__()

        self.mha1 = MultiHeadSelfAttention(embed_dim, num_heads)
        self.mha2 = MultiHeadSelfAttention(embed_dim, num_heads)

        self.ffn = tf.keras.Sequential(
            [L.Dense(ff_dim, activation="relu"), L.Dense(embed_dim),]
        )

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, enc_output, training):
        attn1 = self.mha1(inputs)  # (batch_size, target_seq_len, d_model)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(inputs + attn1)

        if enc_output is not None:
            attn2 = self.mha2(enc_output)  # (batch_size, target_seq_len, d_model)
            attn2 = self.dropout2(attn2, training=training)
            out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)
        else:
            attn2 = self.mha2(inputs)  # (batch_size, target_seq_len, d_model)
            attn2 = self.dropout2(attn2, training=training)
            out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

        ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

        return out3

In [20]:
class TokenAndPositionEmbedding(L.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = L.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = L.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [21]:
nnodes = train_inputs.shape[-2]          # Number of nodes in the graphs
nfeatures = train_inputs.shape[-1]          # Original feature dimensionality

def cbr(x, out_layer, kernel, stride, dilation):
        x = L.Conv1D(out_layer, kernel_size=kernel, dilation_rate=dilation, strides=stride, padding="same")(x)
        # x = L.BatchNormalization()(x)
        x = L.Activation("relu")(x)
        return x

def wave_block(x, filters, kernel_size, n):
        dilation_rates = [2**i for i in range(n)]
        x = tf.keras.layers.Conv1D(filters = filters,
                   kernel_size = 1,
                   padding = 'same')(x)
        res_x = x
        for dilation_rate in dilation_rates:
            tanh_out = tf.keras.layers.Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same', 
                              activation = 'tanh', 
                              dilation_rate = dilation_rate)(x)
            sigm_out = tf.keras.layers.Conv1D(filters = filters,
                              kernel_size = kernel_size,
                              padding = 'same',
                              activation = 'sigmoid', 
                              dilation_rate = dilation_rate)(x)
            x = tf.keras.layers.Multiply()([tanh_out, sigm_out])
            x = tf.keras.layers.Conv1D(filters = filters,
                       kernel_size = 1,
                       padding = 'same')(x)
            res_x = tf.keras.layers.Add()([res_x, x])
        return res_x

def gru_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
                                tf.keras.layers.GRU(hidden_dim,
                                dropout=dropout,
                                return_sequences=True,
                                kernel_initializer = 'orthogonal'))

def lstm_layer(hidden_dim, dropout):
    return tf.keras.layers.Bidirectional(
                                tf.keras.layers.LSTM(hidden_dim,
                                dropout=dropout,
                                return_sequences=True,
                                kernel_initializer = 'orthogonal'))

def mcrmse_loss(y_true, y_pred):
    colwise_mse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
    return tf.reduce_mean(tf.sqrt(colwise_mse), axis=1)

def mcrmse(y_true, y_pred):
    diff = tf.reshape(y_true[:,:,:5] - y_pred, [-1, 5])
    colwise_mse = tf.reduce_mean(tf.square(diff), axis=0)
    return tf.reduce_mean(tf.sqrt(colwise_mse))


def competition_metric(y_true, y_pred):
    total = 0
    for i in range(5):
        rmse = np.mean((y_pred[:,:,i] - y_true[:,:,i])**2)**0.5
        print('{:12s}: {:.4f}'.format(target_cols[i], rmse))
        if target_cols[i] in ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']:
            total += rmse/3
    print('total:', total)

def build_model(model_num=0,seq_len=130, pred_len=68, dropout=0.4,
                embed_dim=512, hidden_dim=256):
    
    X_in = tf.keras.layers.Input(shape=(seq_len, nfeatures))
    A_in = tf.keras.layers.Input(shape=(seq_len, seq_len))
    embed = tf.keras.layers.Dense(embed_dim, activation='linear')(X_in)    
        
    gc1 = GraphAttention(embed_dim, activation='relu', dropout_rate=0.1)([X_in, A_in])
    gc1 = tf.keras.layers.Dropout(.2)(gc1)
    gc1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(gc1 + embed)
    
    gc2 = GraphAttention(embed_dim, activation='relu', dropout_rate=0.1)([gc1, A_in])
    gc2 = tf.keras.layers.Dropout(.2)(gc2)
    gc2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(gc2 + gc1)
    
    reshaped = tf.keras.layers.LayerNormalization(epsilon=1e-6)(gc2 + embed)
    

    encoder_block1 = EncoderBlock(reshaped.shape[2], 4, 32)
    encoder_block2 = EncoderBlock(reshaped.shape[2], 4, 64)
    encoder_block3 = EncoderBlock(reshaped.shape[2], 4, 128)
    
    hidden = encoder_block1(reshaped)
    hidden = encoder_block2(hidden)
    hidden = encoder_block3(hidden)
    
    if model_num == 0:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
  
              
    if model_num == 1:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        
    if model_num == 2:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)

    if model_num == 3:
        hidden = lstm_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)

    if model_num == 4:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        hidden = lstm_layer(hidden_dim, dropout)(hidden) 
        
    if model_num == 5:
        hidden = gru_layer(hidden_dim, dropout)(reshaped)
        hidden = lstm_layer(hidden_dim, dropout)(hidden)
        hidden = gru_layer(hidden_dim, dropout)(hidden)
        
    
    #only making predictions on the first part of each sequence
    truncated = hidden[:, :pred_len]
    
    out = tf.keras.layers.Dense(5, activation='linear')(truncated)

    model = tf.keras.Model(inputs=[X_in, A_in], outputs=out)

    #some optimizers
    adam = tf.optimizers.Adam()
    
    model.compile(optimizer = adam, loss=mcrmse_loss)
    
    return model

In [22]:
#basic training configuration
FOLDS = 10
EPOCHS = 500
BATCH_SIZE = 64*len(gpus)
VERBOSE = 1

lr = tf.keras.callbacks.ReduceLROnPlateau(verbose=VERBOSE, factor=0.9, patience = 5 )
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, verbose=VERBOSE, mode='auto')

In [ ]:
kfold = KFold(FOLDS, shuffle = True, random_state = 34)

#strategy = tf.distribute.MirroredStrategy()

model_name = 'gtn_aug_snr'

train_ids = train['id'].values

models = [0,1,2,3]

gru_private_preds = [None]*len(models)
gru_public_preds = [None]*len(models)
oof = [None]*len(models)

for model_num in models:
    
    gru_private_preds[model_num] = np.zeros((private_df.shape[0], 130, 5))
    gru_public_preds[model_num] = np.zeros((public_df.shape[0], 107, 5))
    oof[model_num] = np.zeros((train_labels.shape[0], 68, 5))

    for fold, (train_index, val_index) in enumerate(list(kfold.split(train_inputs))[:]):
        print('Fold:', fold, ' model_num:', model_num)
        cp = tf.keras.callbacks.ModelCheckpoint(f'{model_name}_{fold}_{model_num}.h5', monitor='val_loss', verbose=VERBOSE, save_best_only=True,
            save_weights_only=True, mode='auto', save_freq='epoch')
        
        #train_index = train_index[train_snfilter[train_index] == 1]
        train_X = train_inputs[train_index]
        train_y = train_labels[train_index]
        train_A = train_adj[train_index]
        train_W = train_weights[train_index]
        
        if 1:
            aug_indx = np.isin(train_ids_aug, train_ids[train_index])    
            train_X_aug = train_inputs_aug[aug_indx]
            train_y_aug = train_labels_aug[aug_indx]
            train_A_aug = train_adj_aug[aug_indx]
            train_W_aug = train_weights_aug[aug_indx]

            train_X = np.concatenate((train_X, train_X_aug))
            train_y = np.concatenate((train_y, train_y_aug))
            train_A = np.concatenate((train_A, train_A_aug))
            train_W = np.concatenate((train_W, train_W_aug))
        
        if 0:
            train_X_aug = public_inputs[good_public]
            train_A_aug = public_adj[good_public]
            train_W_aug = train_weights[train_index].mean()*np.ones(train_X_aug.shape[0])
            train_y_aug = public_labels

            train_X = np.concatenate((train_X, train_X_aug))
            train_y = np.concatenate((train_y, train_y_aug))
            train_A = np.concatenate((train_A, train_A_aug))
            train_W = np.concatenate((train_W, train_W_aug))    
            
            train_X_aug = private_inputs[good_private]
            train_A_aug = private_adj[good_private]
            train_W_aug = train_weights[train_index].mean()*np.ones(train_X_aug.shape[0])
            train_y_aug = private_labels

            train_X = np.concatenate((train_X, train_X_aug))
            train_y = np.concatenate((train_y, train_y_aug))
            train_A = np.concatenate((train_A, train_A_aug))
            train_W = np.concatenate((train_W, train_W_aug)) 
        
        print(train_X.shape)
        
        val_index = val_index[train_snfilter[val_index] == 1]
        valid_X = train_inputs[val_index]
        valid_y = train_labels[val_index]
        valid_A = train_adj[val_index]
        
        model = build_model(model_num)
        history = model.fit([train_X,train_A], train_y, sample_weight = train_W, validation_data=([valid_X,valid_A],valid_y), 
                            batch_size=BATCH_SIZE, epochs=EPOCHS,callbacks=[lr,cp,es],verbose = VERBOSE)      
        
        model.load_weights(f'{model_name}_{fold}_{model_num}.h5')
        oof[model_num][val_index] = model.predict([valid_X,valid_A])
        
        competition_metric(valid_y, oof[model_num][val_index])
    
        
        if 1:
            #load best model and predict
            gru_short = build_model(model_num, pred_len=107)
            gru_short.load_weights(f'{model_name}_{fold}_{model_num}.h5')
            gru_public_pred = gru_short.predict([public_inputs,public_adj]) / FOLDS
            
            gru_long = build_model(model_num, pred_len=130)
            gru_long.load_weights(f'{model_name}_{fold}_{model_num}.h5')
            gru_private_pred = gru_long.predict([private_inputs, private_adj]) / FOLDS
            
            gru_public_preds[model_num] += gru_public_pred
            gru_private_preds[model_num] += gru_private_pred
      
    print('===================OOF RMSE=================')
    competition_metric(train_labels[train_snfilter == 1], oof[model_num][train_snfilter == 1])
    print('============================================')


print('===================COMBINED OOF RMSE=================')
competition_metric(train_labels[train_snfilter == 1], np.mean(oof, axis=0)[train_snfilter == 1])
print('=====================================================')

Streaming output truncated to the last 5000 lines.
68/68 [==============================] - 11s 156ms/step - loss: 0.9452 - val_loss: 0.2293
Epoch 12/500
68/68 [==============================] - ETA: 0s - loss: 0.9268
Epoch 00012: val_loss did not improve from 0.22694
68/68 [==============================] - 11s 156ms/step - loss: 0.9268 - val_loss: 0.2301
Epoch 13/500
68/68 [==============================] - ETA: 0s - loss: 0.9102
Epoch 00013: val_loss did not improve from 0.22694
68/68 [==============================] - 11s 156ms/step - loss: 0.9102 - val_loss: 0.2302
Epoch 14/500
68/68 [==============================] - ETA: 0s - loss: 0.8924
Epoch 00014: val_loss improved from 0.22694 to 0.21958, saving model to gtn_aug_snr_8_0.h5
68/68 [==============================] - 11s 157ms/step - loss: 0.8924 - val_loss: 0.2196
Epoch 15/500
68/68 [==============================] - ETA: 0s - loss: 0.8735
Epoch 00015: val_loss did not improve from 0.21958
68/68 [==============================

In [ ]:
if 0:
    #plt.plot(history.history['loss'][10:], label='train')
    plt.plot(history.history['val_loss'][10:], label='val')
    plt.yscale('log')
    plt.legend()

In [ ]:
np.save(f'/content/{model_name}_oof', oof)
np.save(f'/content/{model_name}_gru_public_preds', gru_public_preds)
np.save(f'/content/{model_name}_gru_private_preds', gru_private_preds)


#gru_public_preds = np.load(f'preds/{model_name}_gru_public_preds.npy')
#gru_private_preds = np.load(f'preds/{model_name}_gru_private_preds.npy')

gru_private_preds = np.mean(gru_private_preds, axis=0)
gru_public_preds = np.mean(gru_public_preds, axis=0)

preds_gru = []

for df, preds in [(public_df, gru_public_preds), (private_df, gru_private_preds)]:
    for i, uid in enumerate(df.id):
        single_pred = preds[i]

        single_df = pd.DataFrame(single_pred, columns=target_cols)
        single_df['id_seqpos'] = [f'{uid}_{x}' for x in range(single_df.shape[0])]

        preds_gru.append(single_df)

preds_gru_df = pd.concat(preds_gru)

preds_gru_df = preds_gru_df.groupby('id_seqpos').mean().reset_index()


submission = sample_sub[['id_seqpos']].merge(preds_gru_df, on=['id_seqpos'])

#sanity check
submission.head()

submission.to_csv(f'/content/drive/My Drive/Vaccine/submissionTransv2.csv', index=False)
print('Submission saved')

print('Run Time [min]:', round((time.time() - start)/60))